This is google colab file please upload run each cell of the notebook to run it efficently 
NOTE:Time complexity of the problem is 20 minutes for whole run 

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import  math


In [2]:
from google.colab import files
uploaded = files.upload()

Saving parcels.csv to parcels.csv
Saving Portsmouth_6MC.csv to Portsmouth_6MC.csv


In [ ]:
df = pd.read_csv("parcels.csv", sep=',')
df=df[['Latitude','Longitude','sub-depot']]
df = df.set_index('sub-depot', drop=True)
df=df.groupby(['Latitude','Longitude']).size().reset_index(name='demand') 
print(df)
grid = df[['Latitude','Longitude']].to_numpy()

In [42]:
class kMeans_from_scratch:
    def __init__(self, coordinates, tolerance, iter, number_of_facility):
        self.grid = coordinates
        self.N = coordinates.shape[0]
        self.t = tolerance
        self.number_of_facility = number_of_facility
        self.f = None
        self.fac = None
        self.assigned_nodes = []
    
    def distance(self,x,y):
    #print(x)
    #print(y)
        coords_1 = x
        coords_2 = y
        return math.sqrt(((coords_1[0]-coords_2[0])**2)+(coords_1[0]-coords_2[0])**2)

    def initial_facility(self):  # randomly create facility locations inside the grid
        self.fac = np.empty([self.number_of_facility, 2])
        for i in range(self.number_of_facility):
            self.fac[i, 0] = np.random.uniform(
                np.min(self.grid[:, 0]), np.max(self.grid[:, 0]))
            self.fac[i, 1] = np.random.uniform(
                np.min(self.grid[:, 1]), np.max(self.grid[:, 1]))
        return self.fac

    def fac_assignment(self, fac):  # Based on shortest distance facility will be assigned
        assign = [0] * self.N
        for n in range(self.N):
            distance = [0] * self.number_of_facility
            #print(distance)
            for f in range(self.number_of_facility):
                #print(f)
                '''
                distance[f] = math.sqrt(
                    (self.grid[n, 0] - fac[f, 0])**2 + (self.grid[n, 1] - fac[f, 1])**2)
                '''
                #print(self.distance((50.842281,-1.059396),(50.83501046289963,-1.0357983470314722)))
                distance[f]=self.distance((self.grid[n, 0],self.grid[n, 1]),(self.fac[f, 0],self.fac[f, 1]))
                
            assign[n] = np.argmin(distance)
        #print(assign)
        return assign

    def nodes_assignment(self, N, facility, assigned):  # all the assigned nodes to the facility
        n_assign = []
        for asg in range(N):
            if facility == assigned[asg]:
                n_assign.append(asg)
        return n_assign

    def kMeans(self, number_of_facility):
        self.initial_facility()
        for itr in range(iter):
            assign = self.fac_assignment(self.fac)
            for facility in range(number_of_facility):
                x, y = 0, 0
                n_assign = self.nodes_assignment(self.N, facility, assign)
                x, y = np.mean(self.grid[n_assign, 0]), np.mean(
                    self.grid[n_assign, 1])
                self.assigned_nodes.append(self.grid[n_assign].tolist())
                if abs(x - self.fac[facility, 0]) >= self.t:
                    self.fac[facility, 0] = x
                if abs(y - self.fac[facility, 1]) >= self.t:
                    self.fac[facility, 1] = y
        print('100')
        self.f = self.fac
        return self.fac




In [45]:
tol, iter, f = 0.01, 200, 6
kM = kMeans_from_scratch(grid, tol, iter, f)
kM.kMeans(6)
#kM.plot()
print('final facilities:\n', kM.fac)
finaloutput=pd.DataFrame()
for i in range(len(list(kM.assigned_nodes[-kM.number_of_facility:]))):
    pdf = pd.DataFrame(list(kM.assigned_nodes[-kM.number_of_facility:][i]))
    #pdf = pdf.transpose()
    pdf['Facility']=[i+1]*len(list(kM.assigned_nodes[-kM.number_of_facility:][i]))
    finaloutput=pd.concat([finaloutput,pdf], axis=0)
    #print(finaloutput)

100
final facilities:
 [[50.8228415  -1.08017025]
 [50.8477278  -1.04536153]
 [50.80554787 -1.08072857]
 [50.7864063  -1.06170972]
 [50.84204684 -1.043842  ]
 [50.78656507 -1.07646678]]


In [47]:
finaloutput=pd.DataFrame()
for i in range(len(list(kM.assigned_nodes[-kM.number_of_facility:]))):
    pdf = pd.DataFrame(list(kM.assigned_nodes[-kM.number_of_facility:][i]))
    #pdf = pdf.transpose()
    pdf['Facility']=[i+1]*len(list(kM.assigned_nodes[-kM.number_of_facility:][i]))
    finaloutput=pd.concat([finaloutput,pdf], axis=0)
    #print(finaloutput)

In [50]:
print('final facilities:\n', kM.fac)


final facilities:
 [[50.8228415  -1.08017025]
 [50.8477278  -1.04536153]
 [50.80554787 -1.08072857]
 [50.7864063  -1.06170972]
 [50.84204684 -1.043842  ]
 [50.78656507 -1.07646678]]


In [51]:
finaloutput.columns=['Latitude','Longitude','Facility']

In [52]:
finaloutput['Facility']=finaloutput['Facility']-1

In [ ]:
finaloutput

In [54]:
Facilities=[]
for i in kM.fac:
  Facilities.append(i)

In [55]:
from geopy.geocoders import Nominatim
location_name=[]
for i in Facilities:
    geolocator = Nominatim(user_agent="geoapiExercises")
    Latitude = str(i[0])
    Longitude = str(i[1])
    location = geolocator.reverse(Latitude+","+Longitude)
    location_name.append(str(location).split("(")[0])
 
# Display
#print(location_name)
facility_dataframe=pd.DataFrame(Facilities,columns=['Facility_Latitude','Facility_Longitude'])
facility_dataframe['Address']=location_name


In [ ]:
facility_dataframe

In [ ]:
pd.set_option('display.max_colwidth', None)
facility_dataframe['Facility_Representation']=facility_dataframe.index
facility_dataframe

In [58]:
#here we are considering geodesic distance because while comparing all the models we should have same type of objective vale
#NOTE: this is not related to k mean
def distance(x,y):
    #print(x)
    #print(y)
    coords_1 = x
    coords_2 = y
    return geopy.distance.geodesic(coords_1, coords_2).km

In [59]:
dffinal=pd.merge(finaloutput, facility_dataframe, left_on='Facility', right_on='Facility_Representation')
dffinal['distance']=dffinal.apply(lambda row: distance((row['Latitude'],row['Longitude']),(row['Facility_Latitude'],row['Facility_Longitude'])),axis=1)

In [ ]:
dffinal

In [ ]:
objective_value=dffinal['distance'].sum()
objective_value

In [ ]:
import seaborn as sns
x=finaloutput['Facility'].unique()
colour=['burlywood','lime','orange','mediumslateblue','lightcoral','gray']
dictcolour={}
for col in range(0,len(x)):
    dictcolour[col]=colour[col]
fig, ax = plt.subplots()
c=sns.color_palette(colour, n_colors=len(x))
sns.scatterplot(finaloutput['Latitude'], finaloutput['Longitude'], c=finaloutput['Facility'],hue=finaloutput['Facility'],marker="+",palette=c)
plt.title(label="PLot for k mean clustering facility")
for col in range(0,len(x)):
    plt.scatter(Facilities[col][0], Facilities[col][1], marker='*', s=600, c=colour[col])

plt.show()